In [1]:
import pickle
import os.path
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser
from whoosh import qparser
import random
from nltk.corpus import stopwords
import nltk
# import pyttsx3
# import speech_recognition as sr
import glob
# import pydub
import asr.align
# from gtts import gTTS
import os
import soundfile as sf
from python_speech_features import mfcc
from python_speech_features import fbank
# nltk.download('stopwords')
import lshknn
import numpy as np
import fnmatch
import json
from sklearn.neighbors import NearestNeighbors

In [2]:
audioFileLocation = '/home/ubuntu/DevData-AudioBooks/dev-clean/'
queryAudioLocation = '/home/ubuntu/spokenkeywordsearch/GroundTruth/'
encodingLocation = '/home/ubuntu/model_encoding/'

In [18]:
class indexer():
    def __init__(self):
        self.transDict = pickle.load(open('transDict2', 'r'))
        self.convertedDict = pickle.load(open('speechRecConvertDict2', 'r'))
        self.queryList = pickle.load(open('queryList2', 'r'))
        self.queryConvertedDict = pickle.load(open('queryConvertedDict2', 'r'))
        schema = Schema(audio=TEXT(stored=True), content=TEXT)
        if not os.path.exists("index"):
            os.mkdir("index")
        self.index = create_in("index", schema)
        writer = self.index.writer()
        for a in self.transDict:
            writer.add_document(audio=u""+a, content=u""+self.transDict[a])
        writer.commit()
        self.searcher = self.index.searcher()
    
    def compareResults(self, indices, distances, audioPosnDict, queryPosnDict, logfileName, limitk):
        log = open(logfileName, 'a')
        print(log)
        totalQueryCount = 0
        totalTrueResults = 0
        totalMatchingResults = 0
        maxMatchingQuery = ""
        maxMatching = 0
        i = 0
        totalFrames = len(nndistances)
        while(i<totalFrames):
            samePos = []
            while(i+1<totalFrames and queryPosnDict[i]==queryPosnDict[i+1]):
                samePos.append(i)
                i += 1
            if(len(samePos)==0):
                samePos = [i]
#             else:
#                 i -= 1
            totalQueryCount += 1
            log.write("------------------------------------------------")
            query = self.queryList[int(queryPosnDict[i])]
#             print(str(i) + " " + query)
            log.write("Results for query: " + query +"\n")
            query = QueryParser("content", self.index.schema, group=qparser.OrGroup).parse(query)
            results = self.searcher.search(query, limit=limitk)
            list1 = []
            list2 = []
            for res in results:
                list1.append(res['audio'])
            for ind in samePos:
                for k in indices[ind]:
                    if(audioPosnDict[k]):
                        log.write(audioPosnDict[k]+" : ")
                        log.write(self.transDict[str(audioPosnDict[k])] + "\n")
                        list2.append(audioPosnDict[k])
            comm = len(set(list1).intersection(set(list2)))
            totalTrueResults += len(list1)
            if(comm>maxMatching):
                maxMatching = comm
                maxMatchingQuery = query
            totalMatchingResults += comm
            log.write("Matching : {0}".format(comm))
            log.flush()
            i += 1
        print("Total queries: ", totalQueryCount)
        print("Total true finds: ", totalTrueResults)
        print("Total matching results: ", totalMatchingResults)

In [19]:
class knn_mfcc():
    def __init__(self, featureType, stride, windowSize):
        self.featureType = featureType
        self.stride = stride
        self.windowSize = windowSize
        self.knnnbrs = []
        
    def initializeFeatures(self, featLocation, kind):
        features = []
        posnDict = dict()
        totalFileCount = 0
        frameCount = 0
            
        if(self.featureType == "mfcc"):
            for root, dirnames, filenames in os.walk(featLocation):
                for filename in fnmatch.filter(filenames, '*.wav'):
                    totalFileCount += 1
                    with open(os.path.join(root, filename), 'rb') as f:
                        data, samplerate = sf.read(f)
                    if(kind=="audio"):
                        mfcc_feat = np.array(mfcc(data, samplerate))
                    else:
                        mfcc_feat = np.array(mfcc(data, samplerate, nfft=600))
                    mfcclen = len(mfcc_feat)
                    i = 0
                    lastLen = 0
                    while(i+self.windowSize<mfcclen):
                        features.append(mfcc_feat[i:i+self.windowSize].flatten())
                        lastLen = len(mfcc_feat[i:i+self.windowSize].flatten())
                        posnDict[frameCount] = filename.split('.')[0]
                        i += self.stride
                        frameCount += 1
#                     if(i<mfcclen):
#                         features.append(mfcc_feat[i:mfcclen].flatten()  [0]*(lastLen-(mfcclen-i)))
                        
        elif(self.featureType == "encoding"):
            if(kind=="audio"):
                fileType = '*.flac.json'
            else:
                fileType = '*.wav.json'
            for root, dirnames, filenames in os.walk(featLocation):
                for filename in fnmatch.filter(filenames, fileType):
                    totalFileCount += 1
                    curr = np.array(json.load(open(root+filename, 'r')))
                    for c in curr:
                        features.append(c)
                        posnDict[frameCount] = filename.split('.')[0]
                        frameCount += 1
        return features, posnDict
    
    def initKNN(self, k, knnalgorithm, audioFeatures):
        self.knnnbrs = NearestNeighbors(n_neighbors=k, algorithm=knnalgorithm).fit(audioFeatures)
        
    def searchForQuery(self, queryFeatures):
        return self.knnnbrs.kneighbors(queryFeatures)

In [9]:
# Using MFCC Features
knnObj = knn_mfcc("mfcc", 10, 30)
audioFeatures, audioPosnDict = knnObj.initializeFeatures(audioFileLocation, "audio")
print(len(audioFeatures))
print(len(audioPosnDict))
queryFeatures, queryPosnDict = knnObj.initializeFeatures(queryAudioLocation, "query")
print(len(queryFeatures))
print(len(queryPosnDict))

186858
186858
18802
18802


In [11]:
# knnObj.initKNN(10, "ball_tree", audioFeatures)
# print("Done initKNN")

Done initKNN


In [13]:
# nndistances, nnindices = knnObj.searchForQuery(queryFeatures)
# print("Done finding nearest neighbors")

Done finding nearest neighbors


In [16]:
# pickle.dump(audioPosnDict, open('audiomfccPosnDictk10.p', 'w'))
# pickle.dump(queryPosnDict, open('querymfccPosnDictk10.p', 'w'))
# pickle.dump(nnindices, open('indicesatk10.p', 'w'))
# pickle.dump(nndistances, open('distancesatk10.p', 'w'))

In [11]:
nndistances = pickle.load(open("distancesatk10.p", 'r'))
nnindices = pickle.load(open("indicesatk10.p", 'r'))

In [40]:
resultchecker = indexer()

In [41]:
resultchecker.compareResults(nnindices, nndistances, audioPosnDict, queryPosnDict, 'mfccFeaturek10.txt', 10)

<open file 'mfccFeaturek10.txt', mode 'a' at 0x7f945ce13c90>
7 hypnotic 
12 work 
18 friend 
24 among 
33 classes 
41 charcoal 
49 intended 
54 good 
63 delia's 
70 garden 
77 showing 
84 become 
90 whole 
97 broken 
102 pray 
109 foul 
115 daughter 
122 going 
128 neither 
133 quick 
142 encounter 
149 judging 
155 long 
164 depressing 
170 shall 
175 ere 
181 arm 
186 ann 
195 unless 
201 hard 
208 ginger 
215 going 
222 holding 
228 fall 
233 make 
239 mingled 
247 always 
258 illustration 
264 public 
268 i've 
273 play 
281 scoundrel 
289 brown's 
298 besides 
304 try 
312 lunch 
316 yet 
324 troubling 
333 kirkleatham 
340 gods 
348 loose 
354 buried 
360 granted 
364 eight 
370 must 
375 go 
381 must 
387 chief 
394 beds 
399 row 
407 husband 
413 ask 
419 glared 
425 yard 
432 voice 
438 time 
447 officer 
452 keep 
460 imagine 
470 cowboys 
476 hundred 
483 scared 
487 could 
493 leaden 
501 cities 
505 ought 
510 fault 
516 morning 
523 going 
528 door 
538 smiles 
546 lose 


4604 old 
4612 mustering 
4617 go 
4626 excitement 
4632 say 
4637 quite 
4644 suffered 
4650 laughed 
4656 man 
4661 folk 
4668 another 
4673 head 
4680 beginning 
4686 time 
4692 friend 
4698 robert 
4709 survives 
4715 fried 
4723 venus 
4730 narrative 
4737 room 
4743 pestle 
4749 he's 
4754 black 
4761 course 
4769 steering 
4775 sir 
4781 much 
4787 long 
4795 delightful 
4800 good 
4806 even 
4811 said 
4816 like 
4825 astounded 
4830 high 
4836 open 
4842 pan 
4850 glacier 
4860 trousers 
4867 yes 
4876 lectures 
4884 promise 
4892 shimerda 
4900 kaliko 
4911 mysterious 
4918 humph 
4924 one 
4930 sent 
4936 sudden 
4942 order 
4950 quickly 
4958 torture 
4963 coat 
4969 decay 
4975 peter 
4982 courts 
4987 thought 
4994 element 
4998 meet 
5004 fading 
5010 one 
5017 soon 
5024 marble 
5030 wine 
5038 profound 
5045 upon 
5052 soon 
5060 roads 
5065 two 
5075 copiously 
5080 grow 
5088 whisper 
5099 superior 
5106 stew 
5112 carrie 
5118 life 
5123 went 
5128 good 
5137 suffic

9345 equally 
9352 west 
9359 gryce 
9367 years 
9375 delightful 
9381 ushered 
9392 curiosity 
9401 houses 
9413 grandparents 
9421 pinkies 
9427 awful 
9435 amongst 
9440 bow 
9452 surroundings 
9457 three 
9462 led 
9471 deucalion 
9479 charge 
9484 said 
9490 one 
9498 fuchs 
9504 salt 
9516 ingredients 
9524 ladies 
9532 tracing 
9541 encounter 
9549 gentleman 
9555 pressed 
9561 eager 
9566 bread 
9572 one 
9580 marjoram 
9587 piece 
9595 kaliko 
9601 placed 
9610 kirkleatham 
9616 street 
9625 pastime 
9631 placed 
9642 enclosure 
9648 brushed 
9655 never 
9664 mayonnaise 
9671 direct 
9677 see 
9684 begin 
9692 longer 
9702 obliterated 
9709 eyes 
9716 showing 
9723 elbow 
9728 said 
9734 one 
9739 old 
9745 truth 
9753 speaks 
9762 variety 
9769 later 
9777 opinion 
9785 hands 
9791 lady 
9800 gardener's 
9806 wife 
9812 time 
9817 note 
9822 two 
9829 center 
9837 quixote 
9843 home 
9849 pickle 
9859 characters 
9865 mean 
9874 cloves 
9881 dreaming 
9887 wind 
9894 growing 

14065 wished 
14070 minute 
14076 come 
14083 brigade 
14092 approach 
14098 one 
14107 ornamental 
14113 left 
14122 sounds 
14129 conceit 
14137 together 
14142 bride 
14147 knew 
14152 head 
14156 let 
14164 house 
14170 gotten 
14181 intolerable 
14191 egyptian 
14200 troublesome 
14208 children 
14214 sight 
14225 exceedingly 
14231 quiet 
14237 obey 
14243 apple 
14250 finely 
14260 offerings 
14266 came 
14270 though 
14281 importance 
14287 came 
14297 impossible 
14303 found 
14313 fakirs 
14320 finally 
14327 small 
14336 cultivated 
14343 dinners 
14348 aunt 
14354 around 
14361 window 
14366 would 
14371 three 
14376 poor 
14384 bayoneted 
14391 yeast 
14401 exterminate 
14407 seemed 
14418 illustration 
14428 meditates 
14434 coming 
14444 established 
14451 protect 
14458 eggs 
14464 wine 
14470 sarah 
14478 forms 
14485 stated 
14493 therefore 
14498 e 
14504 brushed 
14512 friends 
14518 train 
14525 that's 
14531 early 
14538 defeat 
14544 many 
14553 nourishing 
14560

18183 gwynplaine 
18193 entirely 
18200 kingdom 
18206 sure 
18212 loveth 
18218 seen 
18227 charges 
18233 lady 
18239 cruel 
18244 hell 
18251 massive 
18258 making 
18263 neck 
18271 rags 
18277 woman 
18285 offspring 
18291 turn 
18297 away 
18303 found 
18309 tell 
18319 exception 
18327 dominated 
18337 happiness 
18343 van 
18351 delightful 
18358 midst 
18365 editor 
18372 roof 
18377 dah 
18385 excellent 
18391 came 
18397 one 
18403 side 
18411 ladies 
18416 sat 
18423 face 
18429 tell 
18436 almost 
18447 performance 
18453 shame 
18461 begins 
18470 instantly 
18477 fire 
18482 bill 
18487 poor 
18492 thy 
18500 fellows 
18508 orders 
18516 original 
18523 alarm 
18528 mop 
18537 mayonnaise 
18547 stupirti 
18555 troubling 
18562 touching 
18568 pickle 
18576 stacy 
18584 impara 
18594 surprising 
18603 inquired 
18608 added 
18614 whirled 
18621 sanum 
18630 terminating 
18638 lives 
18644 hon 
18650 many 
18658 assaulted 
18664 mary 
18672 mildly 
18679 corner 
18685 turn

In [6]:
# Using Encoding features
knnEncObj = knn_mfcc("encoding", 10, 30)
audioEncFeatures, audioEncPosnDict = knnEncObj.initializeFeatures(encodingLocation, "audio")
print(len(audioFeatures))
print(len(audioPosnDict))
queryEncFeatures, queryEncPosnDict = knnEncObj.initializeFeatures(encodingLocation, "query")
print(len(queryFeatures))
print(len(queryPosnDict))

190833
190833
2703
2703


In [7]:
# Using Encoding features
knnEncObj.initKNN(10, "ball_tree", audioEncFeatures)
print("Done initKNN")
nnEncDistances, nnEncIndices = knnEncObj.searchForQuery(queryEncFeatures)
print(len(nndistances))
print("Done finding nearest neighbors")
resultchecker = indexer()
resultchecker.compareResults(nnEncIndices, nnEncDistances, audioEncPosnDict, queryEncPosnDict, 'mfccFeaturek10.txt', 10)

Done initKNN


2703
Done finding nearest neighbors


<open file 'mfccFeaturek10.txt', mode 'a' at 0x7f0240c89660>
0 bestia 
1 makes 
2 many 
3 neath 
4 citizen 
5 boys 
6 sounds 
7 problem 
8 dangers 
9 turned 
10 writes 
11 pretty 
12 best 
13 cord 
14 rhein 
15 scoundrel 
16 hell 
17 shiloh 
18 home 
19 anyway 
20 colony 
21 suddenly 
22 exceedingly 
23 know 
24 better 
25 can't 
26 foliage 
27 among 
28 could 
29 fifteen 
30 remain 
31 serve 
32 already 
33 glared 
34 save 
35 rushing 
36 faults 
37 account 
38 temporary 
39 living 
40 daring 
41 man 
42 last 
43 look 
44 god 
45 mother 
46 play 
47 splendour 
48 tis 
49 charmed 
50 buildings 
51 hypnotic 
52 hurried 
53 course 
54 shining 
55 last 
56 line 
57 felt 
58 tom 
59 matter 
60 illustration 
61 prince 
62 often 
63 lady 
64 trot 
65 rubbing 
66 shoulders 
67 growled 
68 give 
69 find 
70 feast 
71 soup 
72 worth 
73 ladies 
74 notable 
75 mark 
76 flushing 
77 door 
78 people 
79 start 
80 bring 
81 oil 
82 morning 
83 nodded 
84 reply 
85 small 
86 net 
87 perils 
88 wealt

714 long 
715 together 
716 approaches 
717 indifferent 
718 rhein 
719 robes 
720 school 
721 natural 
722 frightful 
723 universal 
724 prayed 
725 e 
726 go 
727 bennydeck 
728 vegetable 
729 circumstances 
730 laugh 
731 empire 
732 trying 
733 impara 
734 deposited 
735 century 
736 gave 
737 marble 
738 christians 
739 cities 
740 that's 
741 called 
742 macklewain 
743 lungs 
744 three 
745 there's 
746 era 
747 said 
748 three 
749 consider 
750 massive 
751 see 
752 shared 
753 upon 
754 colonel 
755 scene 
756 buried 
757 beaten 
758 history 
759 estimates 
760 court 
761 chair 
762 danced 
763 meant 
764 waking 
765 began 
766 illustration 
767 ran 
768 human 
769 vaguely 
770 exhibited 
771 listen 
772 reported 
773 importance 
774 getting 
775 yet 
776 chef 
777 fuchs 
778 pop 
779 father 
780 excellent 
781 see 
782 keep 
783 gwynplaine 
784 boys 
785 weapon 
786 replied 
787 told 
788 movables 
789 checked 
790 flog 
791 rose 
792 life 
793 eyelids 
794 aware 
795 glared

1489 wonderful 
1490 many 
1491 heart 
1492 fried 
1493 master 
1494 river 
1495 brigade 
1496 dead 
1497 state 
1498 seasoned 
1499 poor 
1500 white 
1501 dear 
1502 mourning 
1503 think 
1504 going 
1505 suburbs 
1506 full 
1507 remark 
1508 ladies 
1509 riches 
1510 scholar 
1511 invariably 
1512 human 
1513 would 
1514 extraordinary 
1515 shaken 
1516 beyond 
1517 new 
1518 recognizes 
1519 also 
1520 dawes 
1521 strove 
1522 get 
1523 drank 
1524 eye 
1525 quickly 
1526 omitted 
1527 hens 
1528 served 
1529 full 
1530 lives 
1531 strict 
1532 refuge 
1533 carrying 
1534 modoc 
1535 hands 
1536 enough 
1537 late 
1538 gentleman 
1539 stony 
1540 said 
1541 five 
1542 front 
1543 low 
1544 lunch 
1545 table 
1546 gravel 
1547 hope 
1548 let 
1549 sanum 
1550 three 
1551 nourishing 
1552 et 
1553 work 
1554 heels 
1555 become 
1556 field 
1557 ground 
1558 wider 
1559 poor 
1560 long 
1561 howling 
1562 could 
1563 say 
1564 beat 
1565 one 
1566 felt 
1567 believe 
1568 whole 
1569 r

2149 eleven 
2150 spoils 
2151 last 
2152 shame 
2153 fell 
2154 overcome 
2155 bow 
2156 strength 
2157 mowed 
2158 plaiting 
2159 old 
2160 delightful 
2161 lay 
2162 private 
2163 right 
2164 straight 
2165 going 
2166 pure 
2167 step 
2168 fireplace 
2169 along 
2170 direct 
2171 bridge 
2172 chicken 
2173 boss 
2174 whether 
2175 yankees 
2176 except 
2177 said 
2178 asura 
2179 short 
2180 body 
2181 everlasting 
2182 freedom 
2183 wholesale 
2184 knows 
2185 able 
2186 take 
2187 hermit 
2188 cowboys 
2189 already 
2190 pan 
2191 center 
2192 privileges 
2193 friend 
2194 feast 
2195 shimerda 
2196 wind 
2197 much 
2198 room 
2199 zingiber 
2200 little 
2201 carries 
2202 eats 
2203 dear 
2204 may 
2205 going 
2206 without 
2207 wounds 
2208 cultivated 
2209 world 
2210 never 
2211 communities 
2212 story 
2213 lovely 
2214 noirtier 
2215 obey 
2216 homecoming 
2217 good 
2218 best 
2219 brave 
2220 rolls 
2221 person 
2222 bringing 
2223 censure 
2224 catherine 
2225 one 
2226 